In [5]:
import sys
print(f'Interpreter dir: {sys.executable}')
import os

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('../')
    
print(f'Working dir: {os.getcwd()}')
%load_ext autoreload
%autoreload 2

Interpreter dir: /Users/jsg/Documents/GitHub/water_hackathlon/.venv_waterhack/bin/python3
Working dir: /Users/jsg/Documents/GitHub/water_hackathlon


In [6]:
import glob
from pathlib import Path
    
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

# Load smart meter data

For every timestamp, calculate:
- Sum of consumption
- sd and other statistics
- Keep the info from the DMA

---

In [75]:
df_attrib = pd.read_excel("data/raw/AWSmartMeterAttributes.xlsx")
df_attrib.columns
df_attrib

,Unique Ref,DMASHORTNAME,Bill Class grouping,City,Post Code,Meter Location,My Use Registered,My Use Registered date,My Use contacted
0,01E91FB9-C6CA-43D6-8AB2-4724B8E183AA,NEWCENMA,Measured Domestic,NEWMARKET,CB8 9,External,No,NaT,Yes
1,025CC02F-2F7C-4547-B6F7-2305EEAD6E98,NEWCENMA,Commercial,NEWMARKET,CB8 9,External,No,NaT,Yes
2,02ABC9EA-DCAC-4D24-893E-DC242E4BC07F,NEWCENMA,Measured Domestic,NEWMARKET,CB8 9,External,No,NaT,NaN
3,02B6D30E-C99D-4E59-9C5F-0900F7255CE2,NEWCENMA,Measured Domestic,NEWMARKET,CB8 9,External,No,NaT,Yes
4,030FC39D-D3FB-489C-A03A-378C97AE5F9D,NEWCENMA,Measured Domestic,NEWMARKET,CB8 9,External,No,NaT,NaN
...,...,...,...,...,...,...,...,...,...
17637,81C249-s57P-513D-F431,NORFIFMA,Measured Domestic,NORWICH,NR6 7,External,No,NaT,NaN
17638,86I255-y24P-956H-W337,NORW21MA,Measured Domestic,NORWICH,NR6 7,External,No,NaT,NaN
17639,89B266-a57L-226G-Y469,NaN,NaN,NORWICH,NR6 7,Internal,No,NaT,NaN
17640,92B222-209E-581B-G031,NORFIFMA,Measured Domestic,NORWICH,NR6 7,External,No,NaT,NaN


In [47]:
file_list = list(Path('data/raw/SmartMeter/').rglob('*.txt'))
file_list

[PosixPath('data/raw/SmartMeter/SmartMetermar2020.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterJul2018.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterFeb2020.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterNov2017.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterJune2017.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterFeb2019.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterApril2019.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterFeb2018.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterApril2018.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterJan2018.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterMay2019.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterAug2018.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterAug2019.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterMay2018.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterJan2019.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterOct2017.txt'),
 PosixPath('data/raw/SmartMeter/SmartMeterJuly2019.txt'),
 PosixPath('data/raw/Smar

In [76]:
def process_file(f,df_attrib):
    # Load file
    df_file = pd.read_csv(f, parse_dates=['DateTime'])
    
    # Merge with DMA info
    df_m = pd.merge(df_attrib[['DMASHORTNAME','Unique Ref','Bill Class grouping']], df_file,left_on='Unique Ref',right_on='UniqueReference')
    
    # Aggregate
    df_agg = df_m.groupby(['DateTime','DMASHORTNAME','Bill Class grouping']).agg({'Consumption':['sum','mean','max','min','std'],})
    
    # Fix columns
    df = df_agg.reset_index()
    df.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in df.columns]
    
    return df

process_file(file_list[0],df_attrib)

,DateTime,DMASHORTNAME,Bill Class grouping,Consumption_sum,Consumption_mean,Consumption_max,Consumption_min,Consumption_std
0,2020-03-01,BLOFLDMA,Commercial,984,27.333333,278548,-278545,94165.935414
1,2020-03-01,BLOFLDMA,Measured Domestic,6805,5.856282,203291,-203284,17337.480706
2,2020-03-01,BLOFLDMA,Unmeasured Domestic,1181,10.359649,95,0,16.202331
3,2020-03-01,BURYRDMA,Commercial,270,33.750000,255,0,89.552459
4,2020-03-01,BURYRDMA,Measured Domestic,223,4.955556,313326,-313318,94470.137890
...,...,...,...,...,...,...,...,...
24580,2020-04-01,NORW37MA,Measured Domestic,2478,2.344371,65,0,5.710557
24581,2020-04-01,NORW37MA,Unmeasured Domestic,803,5.050314,128,0,14.551871
24582,2020-04-01,SWALSHMA,Commercial,441,10.756098,130,0,29.027384
24583,2020-04-01,SWALSHMA,Measured Domestic,1752,3.627329,341270,-341247,43200.296150


In [78]:
list_df = []
for i,f in enumerate(file_list):
    
    print(f"Processing file {f}, {i}/{len(file_list)}")
    df_file = process_file(f, df_attrib)

    # Append
    list_df.append(df_file)

Processing file data/raw/SmartMeter/SmartMetermar2020.txt, 0/44
Processing file data/raw/SmartMeter/SmartMeterJul2018.txt, 1/44
Processing file data/raw/SmartMeter/SmartMeterFeb2020.txt, 2/44
Processing file data/raw/SmartMeter/SmartMeterNov2017.txt, 3/44
Processing file data/raw/SmartMeter/SmartMeterJune2017.txt, 4/44
Processing file data/raw/SmartMeter/SmartMeterFeb2019.txt, 5/44
Processing file data/raw/SmartMeter/SmartMeterApril2019.txt, 6/44
Processing file data/raw/SmartMeter/SmartMeterFeb2018.txt, 7/44
Processing file data/raw/SmartMeter/SmartMeterApril2018.txt, 8/44
Processing file data/raw/SmartMeter/SmartMeterJan2018.txt, 9/44
Processing file data/raw/SmartMeter/SmartMeterMay2019.txt, 10/44
Processing file data/raw/SmartMeter/SmartMeterAug2018.txt, 11/44
Processing file data/raw/SmartMeter/SmartMeterAug2019.txt, 12/44
Processing file data/raw/SmartMeter/SmartMeterMay2018.txt, 13/44
Processing file data/raw/SmartMeter/SmartMeterJan2019.txt, 14/44
Processing file data/raw/Smart

In [79]:
df_all = pd.concat(list_df)
df_all = df_all.sort_values('DateTime')
df_all

,DateTime,DMASHORTNAME,Bill Class grouping,Consumption_sum,Consumption_mean,Consumption_max,Consumption_min,Consumption_std
0,2016-12-05 00:00:00,NEWCENMA,Measured Domestic,0,0.000000,0,0,0.000000
1,2016-12-05 00:00:00,NEWCENMA,Unmeasured Domestic,0,0.000000,0,0,NaN
2,2016-12-05 00:00:00,NEWSEVMA,Measured Domestic,0,0.000000,0,0,0.000000
3,2016-12-05 00:00:00,NEWSEVMA,Unmeasured Domestic,0,0.000000,0,0,NaN
6,2016-12-05 01:00:00,NEWCENMA,Measured Domestic,271,2.356522,25,0,5.049216
...,...,...,...,...,...,...,...,...
20606,2020-07-27 00:00:00,NEWSTUMA,Unmeasured Domestic,389,5.557143,86,0,11.696197
20607,2020-07-27 00:00:00,NORCRDMA,Commercial,240,21.818182,128,0,46.893109
20608,2020-07-27 00:00:00,NORCRDMA,Measured Domestic,1315,2.408425,74,0,6.735626
20598,2020-07-27 00:00:00,NEWCENMA,Commercial,2013,10.166667,1130,0,82.714833


In [80]:
df_all.to_csv('data/processed/smart_meter_aggregated_all.csv', index=False)

# Load a specific house data and visualize

Something is weird...

In [67]:
df_single = pd.read_csv('data/raw/SmartMeter/SmartMetermar2020.txt', parse_dates=['DateTime'])

In [69]:
df_single1 = df_single[df_single['UniqueReference'] == '86B59325-0E22-4FDF-AF03-4FE2881A1DC1']

In [71]:
df_single1.sort_values('DateTime').head(30)

,UniqueReference,DateTime,Consumption
4301958,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 00:00:00,-4989
4368249,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 00:00:00,0
4368248,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 00:00:00,4989
4301957,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 00:00:00,0
8815672,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 01:00:00,0
8822831,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 01:00:00,0
8822832,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 01:00:00,-4989
8815671,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 01:00:00,4989
4368359,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 02:00:00,13
4301995,86B59325-0E22-4FDF-AF03-4FE2881A1DC1,2020-03-01 02:00:00,-4976
